# auth-proxyのメタデータを登録する

---

IdP-proxyにCoursewareHub(auth-proxy)のメタデータを追加する

## 概要

このNotebookでは「221-CoursewareHubのセットアップ.ipynb」で構築したCoursewareHub環境のauth-proxyからメタデータを取得してIdP-proxyへの登録を行います。

IdP-proxyとCoursewareHubの構築担当者が異なる場合は、IdP-proxy構築担当者がCoursewareHub構築担当者からの依頼を受けてこのNotebookを実行してください。

メタデータ登録の際はIdP-proxyからネットワークを通して直接auth-proxyのメタデータを取得します。そのため事前の準備としてIdP-proxyからauth-proxyにアクセスできるようにファイアウォールなどの設定を変更しておいてください。

## 操作対象の設定

操作対象となるAnsibleのグループ名を指定します。

### UnitGroup名

IdP-proxyの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# target_auth = 'IdPproxy'

target_auth = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

指定されたグループ名で対象となるノードが操作できることを確認します。

In [ ]:
!ansible {target_auth} -m ping

管理者権限での操作ができることを確認します。

In [ ]:
!ansible {target_auth} -b -a 'whoami'

## メタデータの追加

CoursewareHub(auth-proxy)のメタデータをIdP-proxyに登録します。

CoursewareHub(auth-proxy)のFQDNを指定してください。

In [ ]:
# (例)
# master_fqdn = 'hub.example.org'

master_fqdn = 

IdP-proxyからauth-proxyのメタデータが取得できること確認します。

In [ ]:
!ansible {target_auth} -a \
    'curl https://{master_fqdn}/simplesaml/module.php/saml/sp/metadata.php/default-sp'

メタデータをIdP-proxyに登録します。IdP-proxyコンテナの`add_auth_proxy.sh`を実行することでIdP-proxyコンテナの`/var/www/simplesamlphp/metadata/xml/auth-proxies.xml`にメタデータが追加されます。

In [ ]:
# IdP-proxyコンテナ名
IDP_PROXY = "idp-proxy"

!ansible {target_auth} -b -a \
    '/usr/local/bin/docker exec -i {IDP_PROXY} /usr/local/sbin/add_auth_proxy.sh {master_fqdn}'

IdP-proxyにメタデータが登録されたことを確認するために SimpleSAMLphpの設定ページにアクセスします。

In [ ]:
out = !ansible {target_auth} -c local \
    -a 'echo "https://{{{{auth_fqdn}}}}/simplesaml/module.php/core/frontpage_federation.php"'
print(out[1])

上記のセルの実行結果に表示されるリンク先にアクセスした後に「管理者でログイン」で管理者となることでメタデータの登録状況を確認することができます。

> 管理者のパスワードはNotebook「521-IdP-proxyのセットアップ」の「4.5 設定ファイルの更新」で`auth.adminpassword` に設定した値となります。